# Load dataframe from Wikipedia

In [ ]:
import pandas as pd

df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df = df[df['Borough'] != 'Not assigned']
df.reset_index(drop=True, inplace=True)
df.rename(columns={"Postcode": "PostalCode"}, inplace=True)

df = df.groupby('PostalCode', as_index=False).agg({
    "Borough" : 'first',
    "Neighborhood": lambda x: ', '.join(x)    
})

for index, row in df.iterrows():
    if df.loc[index, 'Neighborhood'] == 'Not assigned':
        df.loc[index, 'Neighborhood'] = df.loc[index, 'Borough']

df

# Get the latitude and the longitude coordinates of each neighborhood

In [ ]:
dfGeo = pd.read_csv('http://cocl.us/Geospatial_data')
dfGeo

# Merge df and dfGeo in the dataframe

In [ ]:
dfToronto = pd.merge(left=df, right=dfGeo, how='left', left_on='PostalCode', right_on='Postal Code')
dfToronto.drop(labels='Postal Code', axis=1, inplace=True)
dfToronto